In [1]:
import pickle
import pandas as pd
import gc
from sklearn.metrics import classification_report

In [2]:
X = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_train.ftr')
y = pd.read_feather('../input/amex-imputed-and-1hot-encoded/y_train.ftr')

X = X.set_index('customer_ID')
y = y.set_index('customer_ID')['target']

In [3]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [4]:
import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(X, y)

LGBMClassifier()

In [5]:
preds = model.predict(X)
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(preds, index=y.index, columns=["prediction"]))}')

M = 0.5002695574562435


In [6]:
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92   4153582
           1       0.75      0.76      0.76   1377869

    accuracy                           0.88   5531451
   macro avg       0.84      0.84      0.84   5531451
weighted avg       0.88      0.88      0.88   5531451



In [7]:
del X, y
gc.collect()

76

In [8]:
pickle.dump(model, open('LGBM_Classifier.sav', 'wb'))

In [9]:
X_test_1 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_1.ftr')
X_test_1 = X_test_1.set_index('customer_ID')

In [10]:
X_test_2 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_2.ftr')
X_test_2 = X_test_2.set_index('customer_ID')

In [11]:
preds_1 = pd.DataFrame(model.predict_proba(X_test_1)[:, 1], index=X_test_1.index, columns=['prediction'])
preds_2 = pd.DataFrame(model.predict_proba(X_test_2)[:, 1], index=X_test_2.index, columns=['prediction'])

In [12]:
submission = pd.concat([preds_1, preds_2])

# predictions only need to be for each customer
submission = submission.groupby('customer_ID').agg(['last'])
submission.columns = submission.columns.droplevel(1)

# predictions need to be doubles 
submission['prediction'] = submission['prediction'].astype('double')

# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('baseline_lgbm_classifer_submission.csv', index=False)